<a href="https://colab.research.google.com/github/jairodriguez/AgentGPT/blob/main/webinar_langchain_lex_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ask Lex Agent

**Note**: This notebook is not free to run, you will need to create ~20K OpenAI `text-embedding-ada-002` embedding which do cost money. The Pinecone index can be run within the free tier.

In [ ]:
!pip install -qU datasets pinecone-client[grpc] langchain openai tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.5/578.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.2/443.2 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Set api keys, etc:

In [ ]:
OPENAI_API_KEY = "OPENAI_API_KEY"  # platform.openai.com
PINECONE_API_KEY = "PINECONE_API_KEY"  # app.pinecone.io
PINECONE_ENV = "PINECONE_ENV"

First we download a prebuilt Lex Fridman podcast transcriptions dataset:

In [ ]:
from datasets import load_dataset

data = load_dataset(
    'jamescalam/lex-transcripts',
    split='train'
)
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/jamescalam___json/jamescalam--lex-transcripts-6a9688b7915283fe/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


Dataset({
    features: ['video_id', 'channel_id', 'title', 'published', 'transcript', 'source'],
    num_rows: 499
})

Initialize the `indexer` object.

In [ ]:
data[0]

{'video_id': '_SpptYg_0Rs',
 'channel_id': 'UCSHZKyawb77ixDdsGog4iWA',
 'title': 'Are We Living in a Simulation? with George Hotz and Lex Fridman | AI Podcast Clips',
 'published': datetime.datetime(2019, 8, 29, 13, 9, 2),
 'transcript': " Do you think we're living in a simulation? Yes, but it may be unfalsifiable. What do you mean by unfalsifiable? So if the simulation is designed in such a way that they did like a formal proof to show that no information can get in and out, and if their hardware is designed for anything in the simulation to always keep the hardware in spec, it may be impossible to prove whether we're in a simulation or not. So they've designed it such that it's a closed system, you can't get outside the system? Well maybe it's one of three worlds. We're either in a simulation which can be exploited, we're in a simulation which not only can't be exploited, but like the same thing is true about VMs. A really well designed VM, you can't even detect if you're in a VM or 

The chunks of text in the `transcript` field can be very long so we first need to split these into smaller chunks. To count the size of these chunks we need to count the number of `text-embedding-ada-002` tokens. We can do that using the `tiktoken` tokenizer:

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')  # cl100k base is encoder used by ada-002

# define a length function
def tiktoken_len(text: str) -> int:
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)

In [ ]:
tiktoken_len("here is a random sentence we can get the token length for")

12

Now we use that to initialize a LangChain text splitter:

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=20,  # number of tokens overlap between chunks
    length_function=tiktoken_len,  # token count function
    separators=['\n\n', '.\n', '\n', '.', '?', '!', ' ', '']
)

All we need to do now is iterate through the dataset and split the longer transcripts into smaller chunks using the `text_splitter`.

In [ ]:
from tqdm.auto import tqdm

new_data = []

for row in tqdm(data):
    chunks = text_splitter.split_text(row['transcript'])
    row.pop('transcript')
    for i, text in enumerate(chunks):
        new_data.append({**row, **{'chunk': i, 'text': text}})

  0%|          | 0/499 [00:00<?, ?it/s]

In [ ]:
new_data[0]

{'video_id': '_SpptYg_0Rs',
 'channel_id': 'UCSHZKyawb77ixDdsGog4iWA',
 'title': 'Are We Living in a Simulation? with George Hotz and Lex Fridman | AI Podcast Clips',
 'published': datetime.datetime(2019, 8, 29, 13, 9, 2),
 'source': 'https://youtu.be/_SpptYg_0Rs',
 'chunk': 0,
 'text': "Do you think we're living in a simulation? Yes, but it may be unfalsifiable. What do you mean by unfalsifiable? So if the simulation is designed in such a way that they did like a formal proof to show that no information can get in and out, and if their hardware is designed for anything in the simulation to always keep the hardware in spec, it may be impossible to prove whether we're in a simulation or not. So they've designed it such that it's a closed system, you can't get outside the system? Well maybe it's one of three worlds. We're either in a simulation which can be exploited, we're in a simulation which not only can't be exploited, but like the same thing is true about VMs. A really well designe

In [ ]:
new_data[1]

{'video_id': '_SpptYg_0Rs',
 'channel_id': 'UCSHZKyawb77ixDdsGog4iWA',
 'title': 'Are We Living in a Simulation? with George Hotz and Lex Fridman | AI Podcast Clips',
 'published': datetime.datetime(2019, 8, 29, 13, 9, 2),
 'source': 'https://youtu.be/_SpptYg_0Rs',
 'chunk': 1,
 'text': "And if you write code that compiles in a language like that, it is correct by definition. The types check its correctness. So it's possible that the simulation is written in a language like this, in which case, you know. Yeah, but that can't be sufficiently expressive a language like that. Oh, it can. It can be? Oh, yeah. Okay. Well, so, all right, so. The simulation doesn't have to be Turing complete if it has a scheduled end date. Looks like it does, actually, with entropy. I mean, I don't think that a simulation that results in something as complicated as the universe would have a form of proof of correctness, right? It's possible, of course. We have no idea how good their tooling is, and we have no

We need to encode all of these records and store them in a vector DB for later retrieval. To initialize the embedding model we can do:

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

And to initialize the vector DB where we'll be storing the embeddings we do:

In [ ]:
import pinecone

pinecone.init(
    api_key=PINECONE_API_KEY,  # app.pinecone.io
    environment=PINECONE_ENV  # next to API key in console
)

index_name = "pod-gpt"

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name, dimension=1536
    )

index = pinecone.Index(index_name)

Now we loop through and populate our new index:

In [ ]:
batch_size = 100

for i in tqdm(range(0, len(new_data), batch_size)):
    # get end of batch
    i_end = min(len(new_data), i+batch_size)
    # get batch of records
    metadatas = new_data[i:i_end]
    ids = [f"{meta['video_id']}-{meta['chunk']}" for meta in metadatas]
    texts = [meta['text'] for meta in metadatas]
    xc = embeddings.embed_documents(texts)
    to_upsert = zip(ids, xc, metadatas)
    # now add to Pinecone vec DB
    index.upsert(vectors=to_upsert)

  0%|          | 0/362 [00:00<?, ?it/s]

Our vector index is populated and we can move onto the query and interaction with the agent. For this we actually reinitialize the vector db component *via LangChain*:

In [ ]:
from langchain.vectorstores import Pinecone

vectordb = Pinecone(
    index=index,
    embedding_function=embeddings.embed_query,
    text_key="text"
)

Initialize `gpt-3.5-turbo` chat model:

In [ ]:
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name='gpt-3.5-turbo'
)

We then initialize the QA retrieval object using our `llm` and the `vectordb.as_retriever()`:

In [ ]:
from langchain.chains import RetrievalQA

retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

One additional thing we have here is the `chain_type="stuff"`. There are two options here, `"stuff"` or `"map_reduce"`. The `map_reduce` option essentially summarizes returned documents, whereas the `stuff` option just returns the retrieved documents as is.

The `retriever` is ready and can be used by us like this. However, we need to convert it into a `Tool` to be used by our conversational agent. To do that we need the `retriever` itself, a tool description, and a tool name. We use these to initialize the tool like so:

In [ ]:
tool_desc = """Use this tool to answer user questions using Lex
Fridman podcasts. If the user states 'ask Lex' use this tool to get
the answer. This tool can also be used for follow up questions from
the user."""

In [ ]:
from langchain.agents import Tool

tools = [Tool(
    func=retriever.run,
    description=tool_desc,
    name='Lex Fridman DB'
)]

With that, we're ready to initialize the conversational agent. As it is a *conversational* agent, it does need some form of [conversational memory](https://www.pinecone.io/learn/langchain-conversational-memory/). For this we will use the `ConversationBufferWindowMemory` option, which will *remember* the previous `k` interactions between the user and the AI.

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",  # important to align with agent prompt (below)
    k=5,
    return_messages=True
)

In [ ]:
from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent='chat-conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=2,
    early_stopping_method="generate",
    memory=memory,
)

Important items in `agent` parameter:

* `chat-conversational`: for chatbots with conversational memory.
* `react`: refers to the ReAct framework.
* `description`: because the LLM relies on the tool description to decide which tool to use.

### Conversational Agent Prompt

The prompt of the conversational agent is fairly complex. Let's create it then break it down.

In [ ]:
conversational_agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, A

In [ ]:
sys_msg = """You are a helpful chatbot that answers the user's questions.
"""

prompt = conversational_agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
conversational_agent.agent.llm_chain.prompt = prompt

We can see the prompt template like so:

In [ ]:
conversational_agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Lex Fridman DB: Use this tool to answer user questions using Lex\nFridman podcasts. If the user states \'ask Lex\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTION

The conversational agent prompt is defined by the `ChatPromptTemplate`. Let's break it down:

In [ ]:
conversational_agent.agent.llm_chain.prompt.input_variables

['input', 'chat_history', 'agent_scratchpad']

 This prompt template contains *three* `input_variables`, those are:

* `input`: The new user input to the chatbot, i.e. our prompt/query.

* `chat_history`: We defined this above in the `ConversationBufferWindowMemory` definition.

* `agent_scratchpad`: This is where we store the thoughts of the LLM as it is deciding which tools to interact with and *how* to interact with them.

These `input_variables` are fed into the `messages` contained within the prompt template, let's see what we have there:

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Lex Fridman DB: Use this tool to answer user questions using Lex\nFridman podcasts. If the user states \'ask Lex\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTIONS\n----------------------------\n\nWhen responding to me please, please output a response in one of two formats:\n\n**Option 1:**\n


It's a little hard to see here, but there are **three** components in `messages`. Those are:

* `SystemMessagePromptTemplate`

* `MessagesPlaceholder`

* `HumanMessagePromptTemplate`

Let's start with the first item, the `SystemMessage`:

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={})

In [ ]:
print(
    conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template
)

You are a helpful chatbot that answers the user's questions.



That is our initial system message that we set earlier with the `sys_msg`. There's not much to say about this other than it is used to "prime" (set the initial objective of) the model.

Next we have the `MessagesPlaceholder`:

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[1]

MessagesPlaceholder(variable_name='chat_history')

We can see from `'chat_history'` (this must align to the `memory_key` from the `ConversationBufferWindowMemory` initialized earlier) that this is where the previous messages of the conversation will be fed into the LLM.

The format of this input is set by the type of conversational memory being used, which in this case is the `ConversationBufferWindowMemory`.

Finally, we have the `HumanMessagePromptTemplate`:

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[2]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Lex Fridman DB: Use this tool to answer user questions using Lex\nFridman podcasts. If the user states \'ask Lex\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTIONS\n----------------------------\n\nWhen responding to me please, please output a response in one of two formats:\n\n**Option 1:**\nUse this if you want the human to use a tool.\nMarkdown code snippet formatted in the following schema:\n\n```json\n{{\n    "action": string \\ The action to take. Must be one of Lex Fridman DB\n    "action_input": string \\ The input to the action\n}}\n```\n\n**Option #2:**\nUse this if you want to respond directly

In [ ]:
print(
    conversational_agent.agent.llm_chain.prompt.messages[2].prompt.template
)

TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:

> Lex Fridman DB: Use this tool to answer user questions using Lex
Fridman podcasts. If the user states 'ask Lex' use this tool to get
the answer. This tool can also be used for follow up questions from
the user.

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me please, please output a response in one of two formats:

**Option 1:**
Use this if you want the human to use a tool.
Markdown code snippet formatted in the following schema:

```json
{{
    "action": string \ The action to take. Must be one of Lex Fridman DB
    "action_input": string \ The input to the action
}}
```

**Option #2:**
Use this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:

```json
{{
    "action": "Final Answer",
    "action_input": string \ You should put 

This is the most interesting component. First, we have a single `input` — the user's query/prompt. But before this we see a lot of text, the majority of this is the setup for the LLM to be able to use any tools that we've passed to the conversational agent.

In our case, there is just one tool, the `Lex Fridman DB` tool that we defined earlier. We can also see the tool description that we defined. The LLM will use this tool description to figure out which tool (if any) it should use.

## Having a Conversation

Let's begin our conversation. We'll start as any typical conversation begins:

In [ ]:
conversational_agent("hi how are you")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!"
}

> Finished chain.


{'input': 'hi how are you',
 'chat_history': [],
 'output': "I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!"}

Looks good. We should note that there is this **AgentExecutor chain** thing. Where we can see an `"action"` and an `"action_input"`. It is here where the agent is deciding whether it should use a tool.

Here we see the agent decides on `"action": "Final Answer"`, meaning no tool is required. Therefore, it just uses the LLM as per usual to generate an answer. That answer can be seen in `"I'm just a chatbot, I don't have feelings, but thanks for asking! How can I assist you today?"`.

What if we mention the words `"ask lex"`?

In [ ]:
conversational_agent("ask lex about the future of ai")



> Entering new AgentExecutor chain...
```json
{
    "action": "Lex Fridman DB",
    "action_input": "What is the future of AI?"
}
```
Observation: The future of AI is both exciting and terrifying. Many believe that we are on the path to creating superintelligent AGI systems that will surpass the collective intelligence of the human species by many orders of magnitude. This could lead to innumerable applications that will empower humans to create, to flourish, to escape widespread poverty and suffering, and to succeed in the pursuit of happiness. However, it is also terrifying because of the power that superintelligent AGI wields to destroy human civilization, intentionally or unintentionally. The responsibility of AI systems to help millions of people and the ethical considerations of creating meaningful experiences with systems that are faking it before they make it are critical questions that need to be addressed. The development of AGI is a dream for many, but there are concerns a

{'input': 'ask lex about the future of ai',
 'chat_history': [HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={})],
 'output': "I'm sorry, but I don't have any record of your last comment. Could you please repeat it?"}

Great, we can see that the first thing the agent did was default to the `"Lex Fridman DB"` tool. The input to that tool was generated by the LLM, and is `"What did Lex Fridman say about the future of AI?"`.

This input is then passed into the `Lex Fridman DB` tool and the output observation of the LLM (after it has read all of the information returned by our vector DB is returned to our agent. From this observation the agent moves on to the `"Final Answer"` action, giving us the output.

In [ ]:
conversational_agent("what does he think about space exploration?")



> Entering new AgentExecutor chain...
{
    "action": "Lex Fridman DB",
    "action_input": "What does Lex Fridman think about space exploration?"
}
Observation: Lex Fridman believes that space exploration is a beautiful idea and that humans are explorers by nature. He also agrees with Elon Musk's pragmatic view that becoming a multi-planetary species is necessary for our long-term survival. Lex is excited about the possibility of going to Mars, colonizing it, and exploring outside the solar system. He believes that space exploration can inspire people and lead to scientific breakthroughs.
Thought:{
    "action": "Final Answer",
    "action_input": "Lex Fridman believes that space exploration is a beautiful idea and that humans are explorers by nature. He also agrees with Elon Musk's pragmatic view that becoming a multi-planetary species is necessary for our long-term survival. Lex is excited about the possibility of going to Mars, colonizing it, and exploring outside the solar syste

{'input': 'what does he think about space exploration?',
 'chat_history': [HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={}),
  HumanMessage(content='ask lex about the future of ai', additional_kwargs={}),
  AIMessage(content="I'm sorry, but I don't have any record of your last comment. Could you please repeat it?", additional_kwargs={})],
 'output': "Lex Fridman believes that space exploration is a beautiful idea and that humans are explorers by nature. He also agrees with Elon Musk's pragmatic view that becoming a multi-planetary species is necessary for our long-term survival. Lex is excited about the possibility of going to Mars, colonizing it, and exploring outside the solar system. He believes that space exploration can inspire people and lead to scientific breakthroughs."}